# MNIST 模型强化

## 步骤1：载入 MNIST 手写阿拉伯数字资料

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

# 载入 MNIST 手写阿拉伯数字资料
(x_train, y_train),(x_test, y_test) = mnist.load_data()

## 步骤2：改用 CNN 模型

In [2]:
# 建立模型
from tensorflow.keras import layers
import numpy as np

input_shape=(28, 28, 1)
# 增加一维在最后面
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# CNN 模型
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

# 设定优化器(optimizer)、损失函数(loss)、效能衡量指标(metrics)的类别
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 步骤3：资料增补(Data Augmentation)

In [3]:
# 参数设定
batch_size = 1000
epochs = 5

# 资料增补定义
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,         # 特征缩放
        rotation_range=10,      # 旋转 10 度
        zoom_range=0.1,         # 拉远/拉近 10%
        width_shift_range=0.1,  # 宽度偏移  10%
        height_shift_range=0.1) # 高度偏移  10%

# 增补资料，进行模型训练
datagen.fit(x_train)
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
          validation_data=datagen.flow(x_test, y_test, batch_size=batch_size), verbose=2,
          steps_per_epoch=x_train.shape[0]//batch_size)

Epoch 1/5
60/60 - 42s - loss: 1.4652 - accuracy: 0.5203 - val_loss: 0.6872 - val_accuracy: 0.8023
Epoch 2/5
60/60 - 30s - loss: 0.5888 - accuracy: 0.8162 - val_loss: 0.3435 - val_accuracy: 0.9026
Epoch 3/5
60/60 - 31s - loss: 0.3933 - accuracy: 0.8806 - val_loss: 0.2477 - val_accuracy: 0.9300
Epoch 4/5
60/60 - 36s - loss: 0.3136 - accuracy: 0.9033 - val_loss: 0.2013 - val_accuracy: 0.9409
Epoch 5/5
60/60 - 36s - loss: 0.2740 - accuracy: 0.9163 - val_loss: 0.1754 - val_accuracy: 0.9486


In [4]:
# 评分(Score Model)
score=model.evaluate(x_test, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model.metrics_names[i]}: {score[i]:.4f}')

loss: 15.9031
accuracy: 0.9596


## 步骤4：测试自行绘制的数字

In [5]:
# 使用小画家，绘制 0~9，实际测试看看
from skimage import io
from skimage.transform import resize
import numpy as np

# 读取影像并转为单色
uploaded_file = './myDigits/9.png'
image1 = io.imread(uploaded_file, as_gray=True)

# 缩为 (28, 28) 大小的影像
image_resized = resize(image1, (28, 28), anti_aliasing=True)    
X1 = image_resized.reshape(1,28, 28, 1) #/ 255

# 反转颜色，颜色0为白色，与 RGB 色码不同，它的 0 为黑色
X1 = np.abs(1-X1)

# 预测
predictions = np.argmax(model.predict(X1), axis=-1)
print(predictions)

[9]
